### 통합 코드_v2

In [1]:
import os
import joblib
import pickle
from dotenv import load_dotenv
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from openai import OpenAI

# ------------------------------------------
# 0. 환경 설정 및 OpenAI API 초기화
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
client = None
try:
    client = OpenAI(api_key=OPENAI_API_KEY)
    print("OpenAI 컴리언트 처리 완료")
except Exception as e:
    print("OpenAI 컴리언트 처리 실패:", e)

# ------------------------------------------
# 1. MBTI 예측 모델 및 토큰나이저 로딩
mbti_model = load_model("mbti_predictor.h5")
with open("mbti_tokenizer.pkl", "rb") as f:
    mbti_tokenizer = pickle.load(f)
MAX_LEN_MBTI = 100

idx2mbti = {
    0: "INFP", 1: "ENFP", 2: "INFJ", 3: "ENFJ",
    4: "INTP", 5: "ENTP", 6: "INTJ", 7: "ENTJ",
    8: "ISFP", 9: "ESFP", 10: "ISTP", 11: "ESTP",
    12: "ISFJ", 13: "ESFJ", 14: "ISTJ", 15: "ESTJ"
}

def preprocess_mbti(text, tokenizer):
    seq = tokenizer.texts_to_sequences([text])
    return pad_sequences(seq, maxlen=MAX_LEN_MBTI, padding='post')

def predict_from_texts(user_inputs):
    combined_text = " ".join(user_inputs)
    x = preprocess_mbti(combined_text, mbti_tokenizer)
    pred = mbti_model.predict(x)[0]
    idx = pred.argmax()
    return idx2mbti.get(idx, "INFP")

# ------------------------------------------
# 2. 감정 분석 모델
emotion_model = joblib.load("emotion_model.pkl")
vectorizer_emotion = joblib.load("emotion_vectorizer.pkl")

def analyze_sentiment(text):
    x = vectorizer_emotion.transform([text])
    pred = emotion_model.predict(x)
    return pred[0]

# ------------------------------------------
# 3. 상담 모델 (keras)
counsel_model = load_model("counsel_model_keras.h5")
with open("keras_tokenizer.pkl", "rb") as f:
    counsel_tokenizer = pickle.load(f)
MAX_LEN_COUNSEL = 50

def preprocess_counsel(text):
    seq = counsel_tokenizer.texts_to_sequences([text])
    return pad_sequences(seq, maxlen=MAX_LEN_COUNSEL, padding='post')

def generate_counseling_response(user_input, mbti=None, song=None):
    x = preprocess_counsel(user_input)
    pred = counsel_model.predict(x)[0]
    response_idx = pred.argmax()
    response_map = {
        0: "당신의 고문을 잘 이해했습니다. 조금만 더 힘내세요!",
        1: "지금 상황이 어렵겠지만 분명히 좋은 결과가 있을 것이에요.",
        2: "긍정적인 말지로 상황을 바라보는 것이 중요합니다."
    }
    base_response = response_map.get(response_idx, "자세히 보여주세요.")
    if song:
        return f"{base_response}\n\n[추천 가상] {song} 을(를) 들어보면 조금만 더 힘이 될 것 같아요."
    return base_response

# ------------------------------------------
# 4. Tool 정의
def predict_mbti_tool(user_inputs: list) -> str:
    if len(user_inputs) < 5:
        return "문장은 최소 5개 이상 필요합니다."
    return predict_from_texts(user_inputs)

def predict_emotion_tool(user_text: str) -> str:
    return analyze_sentiment(user_text)

def generate_counsel_tool(user_text: str, mbti: str, song: str) -> str:
    return generate_counseling_response(user_text, mbti, song)

mbti_tools = [
    {
        "type": "function",
        "function": predict_mbti_tool,
        "name": "predict_mbti",
        "description": "5개 문장을 기본으로 MBTI 성격 유형을 예측합니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "user_inputs": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": "사용자가 입력한 5개의 문장 리스트"
                }
            },
            "required": ["user_inputs"]
        }
    },
    {
        "type": "function",
        "function": predict_emotion_tool,
        "name": "predict_emotion",
        "description": "사용자 입력 문장에서 감정을 분류합니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "user_text": {
                    "type": "string",
                    "description": "사용자가 입력한 텍스트"
                }
            },
            "required": ["user_text"]
        }
    },
    {
        "type": "function",
        "function": generate_counsel_tool,
        "name": "generate_counsel",
        "description": "MBTI와 감정 기반으로 상담 응답을 생성합니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "user_text": {"type": "string", "description": "사용자의 고문 문장"},
                "mbti": {"type": "string", "description": "예측된 MBTI 유형"},
                "song": {"type": "string", "description": "추천 음악 제목"}
            },
            "required": ["user_text", "mbti", "song"]
        }
    }
]

# ------------------------------------------
# 테스트 프로세시어
if __name__ == '__main__':
    user_inputs = [
        "나는 호랑에 있는 시간이 편하다.",
        "다른 사람을 도움을 줄 때 보람을 느낄 수 있어.",
        "가끔 감정 기복이 심한 편이야.",
        "계획보다는 즉흥적인 결정을 많이 해.",
        "사람과 깊은 대화를 나누는 것이 좋아."
    ]
    mbti = predict_mbti_tool(user_inputs)
    print(f"[MBTI 예측 결과] {mbti}")

    user_text = "지금 리비니에 대한 고문이 많아서 불안해."
    emotion = predict_emotion_tool(user_text)
    print(f"[\uac10정 \ubd84석 \uacb0\uacfc] {emotion}")

    counseling = generate_counsel_tool(user_text, mbti, "IU - 밤편지")
    print("[\uc0c1\ub2f4 \uc751\ub2f5]")
    print(counseling)

OpenAI 컴리언트 처리 완료


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'mbti_predictor.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)